In [ ]:
using Revise
using Paint

In [ ]:
using Serialization
using Images, ImageShow
using Plots
using Evolutionary
using BlackBoxOptim
using StaticArrays
using ImageFeatures
using IntervalSets
using ReinforcementLearning
using ReinforcementLearningBase
using ReinforcementLearningZoo
using Combinatorics
using StatsBase

In [ ]:
target = float.(load("../lisa.png"))

In [ ]:
hist1 = Serialization.deserialize("../output/simresult/simlog_100-prims_100000-batch_3-epoch_100-refine.bin")
hist2 = Serialization.deserialize("../output/simresult/simlog_100-prims_10x-blackbox_5000-batch_5-epoch_250-refine.bin")
hists = [hist1, hist2]
minlen = minimum(map(hist -> length(hist.history), hists))

In [ ]:
function drawstate(state, size)
    initial = zeros(RGB{Float32}, size) .+ state.background
    for k in eachindex(state.shapes)
        draw!(initial, state.shapes[k], state.current_colours[k], RasterAlgorithmScanline())
    end
    initial
end

In [ ]:
gr()
plots = []
anim_singles = @animate for i = 1:minlen+250
    ival = min(i, minlen)
    frame = []
    for hist in hists
        res = hist.history[ival]
        drawsize = size(target)
        img = drawstate(res, drawsize)
        plt = plot(img, size=drawsize, showaxis=false, ticks=false, title=string(ival-1))
        push!(frame, plt)
    end
    push!(plots, plot(frame..., layout=(1,length(frame)), size=(512,256)))
end
gif(anim_singles, "/tmp/tmp.gif", fps=12, show_msg=false)

In [ ]:
plotly()

function losses(hist)
    vals = [state.best for state in hist.history]
    diffs = [vals[i] - vals[i - 1] for i = 2:length(vals)]
    vals, diffs
end

ps = collect(map(first, map(losses, hists)))
ps[1] = ps[1] ./ 3.0
plot(ps)